In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os
import logging

os.environ["HF_HOME"] = "../models/"
os.environ["TORCH_HOME"] = "../models/"
os.environ["TRANSFORMERS_OFFLINE"] = "1"

# logging.basicConfig(level=logging.INFO)

In [3]:
from til_23_finals.services.speaker import NeMoSpeakerIDService
from til_23_finals.utils import load_audio_from_dir
import librosa
import numpy as np
import itertools
from pathlib import Path
from time import sleep

[NeMo W 2023-06-15 02:46:59 optimizers:54] Apex was not found. Using the lamb or fused_adam optimizer will error out.
[NeMo W 2023-06-15 02:47:00 experimental:27] Module <class 'nemo.collections.asr.modules.audio_modules.SpectrogramToMultichannelFeatures'> is experimental, not ready for production and is not fully supported. Use at your own risk.


In [4]:
speaker_service = NeMoSpeakerIDService(
    "models/speakerverification_en_titanet_large.nemo",
    "models/DeepFilterNet3"
)

[NeMo W 2023-06-15 02:47:02 modelPT:161] If you intend to do training or fine-tuning, please call the ModelPT.setup_training_data() method and provide a valid configuration file to setup the train data loader.
    Train config : 
    manifest_filepath: /manifests/combined_fisher_swbd_voxceleb12_librispeech/train.json
    sample_rate: 16000
    labels: null
    batch_size: 64
    shuffle: true
    is_tarred: false
    tarred_audio_filepaths: null
    tarred_shard_strategy: scatter
    augmentor:
      noise:
        manifest_path: /manifests/noise/rir_noise_manifest.json
        prob: 0.5
        min_snr_db: 0
        max_snr_db: 15
      speed:
        prob: 0.5
        sr: 16000
        resample_type: kaiser_fast
        min_speed_rate: 0.95
        max_speed_rate: 1.05
    num_workers: 15
    pin_memory: true
    
[NeMo W 2023-06-15 02:47:02 modelPT:168] If you intend to do validation, please call the ModelPT.setup_validation_data() or ModelPT.setup_multiple_validation_data() method 

[NeMo I 2023-06-15 02:47:02 features:287] PADDING: 16
[NeMo I 2023-06-15 02:47:03 save_restore_connector:249] Model EncDecSpeakerLabelModel was successfully restored from /home/jh/code/til/til-23-finals/models/speakerverification_en_titanet_large.nemo.


fatal: not a git repository (or any of the parent directories): .git


In [5]:
speaker_dir = "data/audio/train"
val_dir = "data/audio/val"

In [6]:
with speaker_service:
    speaker_audio = load_audio_from_dir(speaker_dir)
    for name, (wav, sr) in speaker_audio.items():
        # if all(n.upper() not in name.upper() for n in [us_team, them_team]):
        #     continue

        team, member = name.split("_")[:2]
        speaker_service.enroll_speaker(wav, sr, team_id=team, member_id=member)

In [7]:
def evaluate(wav, sr, team_a, team_b, correct):
    d_raw_a, d_clean_a = speaker_service.identify_speaker(wav, sr, team_id=team_a, return_all=True)
    d_raw_b, d_clean_b = speaker_service.identify_speaker(wav, sr, team_id=team_b, return_all=True)
    d_raw = {**d_raw_a, **d_raw_b}
    d_clean = {**d_clean_a, **d_clean_b}

    raw = np.array(list(d_raw.values()))
    clean = np.array(list(d_clean.values()))
    raw_sd = (raw.max() - raw.mean()) / raw.std()
    clean_sd = (clean.max() - clean.mean()) / clean.std()
    raw_ans = "_".join(max(d_raw, key=d_raw.get))
    clean_ans = "_".join(max(d_clean, key=d_clean.get))

    print(f"{team_a} vs {team_b}, correct: {correct}")
    print(f"[{'✓' if raw_ans == correct else 'X'} Raw] {raw_ans} {raw.max()*100:.1f}% {raw_sd:.2f}σ")
    print(f"[{'✓' if clean_ans == correct else 'X'} Clean] {clean_ans} {clean.max()*100:.1f}% {clean_sd:.2f}σ")
    return raw_ans != correct, raw_ans, raw_sd, clean_ans != correct, clean_ans, clean_sd

In [8]:
vals = list(Path(val_dir).glob("*.wav"))

In [9]:
total = 0
raw_false = 0
raw_wrong_ans = []
clean_false = 0
clean_wrong_ans = []
raw_sds = []
clean_sds = []
with speaker_service:
    for audio1, audio2 in itertools.permutations(vals, 2):
        total += 2
        team_1, member1 = audio1.stem.split("_")[:2]
        team_2, member2 = audio2.stem.split("_")[:2]
        wav1, sr1 = librosa.load(audio1, sr=None)
        wav2, sr2 = librosa.load(audio2, sr=None)

        raw_wrong, raw_ans, raw_sd, clean_wrong, clean_ans, clean_sd = evaluate(wav1, sr1, team_1, team_2, f"{team_1}_{member1}")
        if raw_wrong:
            raw_false += 1
            raw_wrong_ans.append(raw_ans)
        else:
            raw_sds.append(raw_sd)
        if clean_wrong:
            clean_false += 1
            clean_wrong_ans.append(clean_ans)
        else:
            clean_sds.append(clean_sd)
        print()

        raw_wrong, raw_ans, raw_sd, clean_wrong, clean_ans, clean_sd = evaluate(wav2, sr2, team_1, team_2, f"{team_2}_{member2}")
        if raw_wrong:
            raw_false += 1
            raw_wrong_ans.append(raw_ans)
        else:
            raw_sds.append(raw_sd)
        if clean_wrong:
            clean_false += 1
            clean_wrong_ans.append(clean_ans)
        else:
            clean_sds.append(clean_sd)
        print()

print(f"Raw False: {raw_false}/{total} ({raw_false/total*100:.2f}%)")
print(f"Raw Mean Sigma: {np.mean(raw_sds):.2f}")
print(f"Raw Wrong Answers: {raw_wrong_ans}")
print(f"Clean False: {clean_false}/{total} ({clean_false/total*100:.2f}%)")
print(f"Clean Mean Sigma: {np.mean(clean_sds):.2f}")
print(f"Clean Wrong Answers: {clean_wrong_ans}")

IMAGINELOSIN vs PALMTREE, correct: IMAGINELOSIN_memberB
[✓ Raw] IMAGINELOSIN_memberB 77.6% 1.93σ
[✓ Clean] IMAGINELOSIN_memberB 74.1% 1.90σ

IMAGINELOSIN vs PALMTREE, correct: PALMTREE_memberD
[✓ Raw] PALMTREE_memberD 77.0% 1.21σ
[✓ Clean] PALMTREE_memberD 77.0% 1.21σ

IMAGINELOSIN vs ANYTHING, correct: IMAGINELOSIN_memberB
[✓ Raw] IMAGINELOSIN_memberB 77.6% 1.85σ
[✓ Clean] IMAGINELOSIN_memberB 74.1% 1.85σ

IMAGINELOSIN vs ANYTHING, correct: ANYTHING_memberC
[✓ Raw] ANYTHING_memberC 89.5% 2.45σ
[✓ Clean] ANYTHING_memberC 90.2% 2.50σ

IMAGINELOSIN vs ACESOFSPADES, correct: IMAGINELOSIN_memberB
[✓ Raw] IMAGINELOSIN_memberB 77.6% 2.42σ
[✓ Clean] IMAGINELOSIN_memberB 74.1% 2.37σ

IMAGINELOSIN vs ACESOFSPADES, correct: ACESOFSPADES_memberC
[✓ Raw] ACESOFSPADES_memberC 73.9% 2.05σ
[✓ Clean] ACESOFSPADES_memberC 76.5% 2.03σ

IMAGINELOSIN vs 10000SGDMRT, correct: IMAGINELOSIN_memberB
[✓ Raw] IMAGINELOSIN_memberB 77.6% 2.25σ
[✓ Clean] IMAGINELOSIN_memberB 74.1% 2.20σ

IMAGINELOSIN vs 10000SGDMR